In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

sys.path.insert(0, os.path.abspath(".."))
sys.path.append(os.path.abspath("../backend"))
sys.path.append(os.path.abspath(""))

In [3]:

from rdflib.plugins.stores.sparqlstore import SPARQLStore
from backend.model import *
from backend.ontology import *
from backend.datasetmatcher import *
from backend.explorative_support import *

base_path = "../data"
onto_path = f"{base_path}/hero-ontology/hereditary_clinical.ttl"


# graph = Graph().parse(onto_path, format="turtle")
# graph.bind("bto", "http://www.semanticweb.org/ontologies/2020/3/bto#")

store = SPARQLStore(
    "http://localhost:7200/repositories/dpedia",
    method="POST_FORM",
    params={"infer": False, "sameAs": False},
)
graph = Graph(store=store)

config = OntologyConfig()

ontology_manager = OntologyManager(config, graph)
dataset_manager = DatasetManager(ontology_manager)
dataset_manager.initialise(glob_path="data/datasets/ALS/**/*.csv")

# ontology_manager.load_full_graph()


-- Loading and merging datasets
0 datasets loaded


In [ ]:
topic_man = TopicModelling(ontology_manager)
topic_man.initialize_topics(force=True)

Some weights of the model checkpoint at dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
llama_load_model_from_file: using device Metal (Apple M2 Pro) - 7615 MiB free
llama_model_loader: loaded meta data with 27 key-value pairs and 292 tensors from /Users/benedikt/.cache/huggingface/hub/models--NousResearch--Hermes-3-Llama-3.1-8B-GGUF/snapshots/307a5dfb59aa38d88b6cfd32f44b8ad7c1da9fb8/./Hermes-3-Llama-3.1-8B.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping

Batches:   0%|          | 0/80 [00:00<?, ?it/s]

2024-11-27 09:47:26,553 - BERTopic - Embedding - Completed ✓
2024-11-27 09:47:26,554 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-27 09:47:33,438 - BERTopic - Dimensionality - Completed ✓
2024-11-27 09:47:33,439 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-27 09:47:33,482 - BERTopic - Cluster - Completed ✓
2024-11-27 09:47:33,485 - BERTopic - Representation - Extracting topics from clusters using representation models.
  0%|          | 0/100 [00:00<?, ?it/s]llama_perf_context_print:        load time =    3197.62 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   427 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     5 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    3446.44 ms /   432 tokens
  1%|          | 1/100 [00:03<05:41,  3.45s/it]Llama.generate: 267 prefix-match hit, re